# Tutorial del Framework Gymnasium y Herramientas de Trabajo proporcionadas por la cátedra

Importamos la biblioteca Gymnassium, que vamos a usar como framework de RL

In [ ]:
!pip3 install cmake gymnasium scipy
import gymnasium as gym

Creamos un ambiente y lo mostramos en pantalla. Para esto definimos una función para imprimir nuestro ambiente.

In [ ]:
# La semilla usada para crear el ambiente
semilla = 1

entorno = gym.make("Taxi-v3", render_mode='ansi').env
entorno.reset(seed = semilla)

# Una funcion de ayuda para imprimir el estado de nuestro mundo
def print_env(estado):
  env_str = estado.render()
  print(env_str.strip())

print_env(entorno)


El rectángulo de color representa el taxi, amarillo cuando va sin pasajero y verde con pasajero.
'|' representa una pared que el taxi no puede cruzar, es decir.
R, G, Y, B son los puntos de interés, es decir, las posibles ubicaciones de recogida y destino. La letra azul representa la ubicación actual de recogida de pasajeros, y la letra púrpura es el destino actual.

Si cambiamos la semilla, cambia el estado del ambiente.

In [ ]:
# Una semilla diferente
semilla = 2

entorno = gym.make("Taxi-v3", render_mode='ansi').env
entorno.reset(seed = semilla)

print_env(entorno)

Exploremos el espacio de estados y de acciones:

In [ ]:
print(f"Espacio de Acciones {entorno.action_space}")
print(f"Espacio de Estados {entorno.observation_space}")

Hay 6 acciones, que corresponden a:
 * 0 = ir al Sur
 * 1 = ir al Norte
 * 2 = ir al Este
 * 3 = ir al Oeste
 * 4 = recoger pasajero
 * 5 = dejar pasajero

Los puntos cardinales siguen la convención Norte hacia arriba. Recoger/dejar al pasajero solo tienen efecto si el taxi está en la misma casilla que el pasajero, y en uno de los puntos de interés.

Nuestro agente deberá elegir la acción a tomar en cada paso. Gymnassium nos expone funciones para esto. Si queremos movernos al sur, por ejemplo:

In [ ]:
semilla = 1
entorno = gym.make("Taxi-v3", render_mode='ansi').env
entorno.reset(seed = semilla)
print_env(entorno)
print()

accion = 0 # Sur
entorno.step(accion)

print_env(entorno)

Ahora estamos listos para programar un agente. Empezando por uno random. Se puede ejecutar el codigo abajo varias veces para ver como cambia en cada ejecución.

In [ ]:
import random 

def episodio_random(semilla_ambiente = 1):
    entorno = gym.make("Taxi-v3", render_mode='ansi').env
    entorno.reset(seed = semilla_ambiente)

    iteraciones = 0
    penalizaciones, recompensa = 0, 0

    marcos = [] # para la animación

    termino = False
    truncado = False

    while not termino and not truncado:
        #  selecciona una acción aleatoria del conjunto de todas las posibles acciones
        accion = entorno.action_space.sample() 
        estado, recompensa, termino, truncado, info = entorno.step(accion)

        # El agente trato de dejar/recoger al pasajero incorrectamente
        if recompensa == -10:
            penalizaciones += 1

        # Put each rendered frame into dict for animation
        marcos.append({
            'marco': entorno.render(),
            'estado': estado,
            'accion': accion,
            'recompensa': recompensa
            }
        )

        iteraciones += 1


    print(f"Iteraciones: {iteraciones}")
    print(f"Penalizaciones: {penalizaciones}")

    return marcos

marcos = episodio_random()

Podemos ver el episodio completo abajo. Notar que seleccionamos la semillia de selector de acciones para que la corrida sea 'buena'.

In [ ]:
from IPython.display import clear_output
from time import sleep
import sys

def print_frames(marcos, delay=0.01):
    for i, marco in enumerate(marcos):
        clear_output()
        print(marco['marco'])
        print(f"Iteracion: {i + 1}")
        print(f"Estado: {marco['estado']}")
        print(f"Accion: {marco['accion']}")
        print(f"Recompensa: {marco['recompensa']}")
        sys.stdout.flush()
        # Aumentar este tiempo para ver mejor la animación
        sleep(delay)

print_frames(marcos)

Ahora queremos programar un agente inteligente, para eso nos vamos a atener a la siguiente interfaz.

In [ ]:
class Agente:
    def elegir_accion(self, estado, max_accion) -> int:
        """Elegir la accion a tomar en el estado actual y el espacio de acciones"""
        pass

    def aprender(self, estado_anterior, estado_siguiente, accion, recompensa):
        """Aprender a partir de la tupla 
            - estado_anterior: el estado desde que se empezó
            - estado_siguiente: el estado al que se llegó
            - accion: la acción que llevo al agente desde estado_anterior a estado_siguiente
            - recompensa: la recompensa recibida en la transicion
        """
        pass

Para nuestro agente aleatorio, esto sería:

In [ ]:
import random

class AgenteAleatorio(Agente):
    def elegir_accion(self, estado, max_accion) -> int:
        # Elige una acción al azar
        return random.randrange(max_accion)

    def aprender(self, estado_anterior, estado_siguiente, accion, recompensa):
        # No aprende
        pass

Poniendolo a jugar:

In [ ]:
import pdb
semilla = 1
entorno = gym.make("Taxi-v3", render_mode='ansi').env

agente = AgenteAleatorio()

iteraciones = 0
penalizaciones, recompensa = 0, 0

marcos = [] # for animation

termino = False
truncado = False
estado_anterior, info = entorno.reset(seed = semilla)
while not termino and not truncado:
    # Le pedimos al agente que elija entre las posibles acciones (0..entorno.action_space.n)
    accion = agente.elegir_accion(estado_anterior, entorno.action_space.n)

    # Realizamos la accion
    estado_siguiente, recompensa, termino, truncado, info = entorno.step(accion)

    # Le informamos al agente para que aprenda
    agente.aprender(estado_anterior, estado_siguiente, accion, recompensa)

    # El agente trato de dejar/recoger al pasajero incorrectamente
    if recompensa == -10:
        penalizaciones += 1

    # Put each rendered frame into dict for animation
    marcos.append({
        'marco': entorno.render(),
        'estado': estado_siguiente,
        'accion': accion,
        'recompensa': recompensa
        }
    )

    estado_anterior = estado_siguiente
    iteraciones += 1

print(f"Iteraciones: {iteraciones}")
print(f"Penalizaciones: {penalizaciones}")

Podemos encapsular lo anterior en una función 

In [ ]:
def ejecutar_episodio(agente, semilla):
    entorno = gym.make("Taxi-v3", render_mode='ansi').env

    iteraciones = 0
    penalizaciones, recompensa = 0, 0

    marcos = [] # for animation

    termino = False
    truncado = False
    estado_anterior, info = entorno.reset(seed = semilla)
    while not termino and not truncado:
        # Le pedimos al agente que elija entre las posibles acciones (0..entorno.action_space.n)
        accion = agente.elegir_accion(estado_anterior, entorno.action_space.n)
        # Realizamos la accion
        estado_siguiente, recompensa, termino, truncado, info = entorno.step(accion)
        # Le informamos al agente para que aprenda
        agente.aprender(estado_anterior, estado_siguiente, accion, recompensa)

        # El agente trato de dejar/recoger al pasajero incorrectamente
        if recompensa == -10:
            penalizaciones += 1

        # Put each rendered frame into dict for animation
        marcos.append({
            'marco': entorno.render(),
            'estado': estado_siguiente,
            'accion': accion,
            'recompensa': recompensa
            }
        )

        estado_anterior = estado_siguiente
        iteraciones += 1
        
    return iteraciones, penalizaciones, marcos


y correrlo varias veces para ver el rendimiento promedio

In [ ]:
agente = AgenteAleatorio()
semilla = 1
num_iteraciones_episodios = []

for i in range(10):
    num_iteraciones, _, _ = ejecutar_episodio(agente, semilla)
    num_iteraciones_episodios += [num_iteraciones]

Y obtener métricas al respecto

In [ ]:
import numpy
import random

print(f"Se realizaron {numpy.mean(num_iteraciones_episodios)} iteraciones, en promedio")

# Entrega 3, Grupo 02 - Aprendizaje por Refuerzos

| Nombre           | C.I     | Email                        |
|----------------|-----------|------------------------------|
| Santiago Alaniz| 5082647-6 | santiago.alaniz@fing.edu.uy  |
| Bruno De Simone| 4914555-0 | bruno.de.simone@fing.edu.uy  |
| María Usuca    | 4891124-3 | maria.usuca@fing.edu.uy      |


## Introduccion


El modelado de la realidad en la cual se entrena al agente es definido por el framework `gymnasium`, escenario `taxi`. Obtenido de la [documentación](https://gymnasium.farama.org/environments/toy_text/taxi/) además de las posibles acciones se tiene que la ubicación del pasajero y los posibles destinos se representan:

**Ubicaciones del pasajero:**
* 0: Rojo
* 1: Verde
* 2: Amarillo
* 3: Azul
* 4: En taxi

**Destinos:**
* 0: Rojo
* 1: Verde
* 2: Amarillo
* 3: Azul

También de la documentación del framework vemos que define los estados con la siguiente ecuación:

```
((taxi_row * 5 + taxi_col) * 5 + passenger_location) * 4 + destination
```

Se observa que parte de lo que define un estado, y por lo cual se podría considerar que el taxi tiene visión global del escenario en el que se encuentra, es la ubicación del pasajero y su destino. Dado que se conoce el destino la cadena de Markov de la cual se quiere conocer la política óptima, sabemos que la misma es no conexa ya que dado cualquier estado para el cual el destino sea `Rojo` nunca se va a poder pasar a un estado para el cual el destino sea `Verde`.

<br>
  <p align="center">
    <img width="400" src="img/diagrama-destinos.png"/>
  </p>
<br>

Además, al saber la ubicación del pasajero se observa que dado un destino `X` y una ubicación `Y` distinta de 4, que significa que el pasajero está en el taxi, solo se puede pasar a un estado que tenga el mismo destino y ubicación O a uno que tenga el mismo destino y ubicación 4. Además, si se tiene ubicación de pasajero 4 nunca va a poder cambiar la ubicación del pasajero.

<br>
  <p align="center">
    <img width="400" src="img/diagrama-ubicacion-pasajero.png"/>
  </p>
<br>

Se ve entonces que si entrenamos en un escenario cuyo destino es `X` luego para cualquier escenario en el cual el destino sea distinto de `X` el taxista no va a saber qué hacer sin entrenamiento ya que nunca pasó por dicho estado ni va a pasar por ningún estado en el cual el destino sea `X`. Lo mismo ocurre al entrenar para un escenario con ubicación inicial del pasajero `Y`, luego al estar en un escenario con ubicación inicial distinta hasta que no recoja al pasajero siempre va a pasar por estados desconocidos.

## Parte 1

Programar las funciones de la clase `AgenteRL`, manteniendo cualquier función adicional necesaria en la misma clase.

### **Clase `AgenteRL`**

Esta implementación está fuertemente inspirada en los conceptos presentados en el libro de Mitchell, *Machine Learning* (1997) y las clases impartidas por el equipo docente.

El agente interactúa con un entorno provisto por el cuerpo docente, donde debemos mover un taxi en un mapa para recoger y dejar pasajeros. El objetivo es que el taxi aprenda la política óptima, es decir, recoger y dejar al pasajero con la menor cantidad de pasos y con la menor cantidad de penalizaciones.

##### **Variables de instancia**:
- `self.gamma`: Un factor de descuento, generalmente denotado como gamma (`γ`), que determina cuánto valor le da el agente a las recompensas futuras en comparación con las inmediatas. Toma como valor constante `0.9`. A diferencia de `self.k` y `self.delta_t`, no cambia con el tiempo, aunque somos conscientes de que agentes más vanguardistas aplican técnicas de perfeccionamiento de este valor a lo largo del tiempo.

- `self.delta_t`: Es una discretización de las acciones tomadas en un episodio. Se inicializa en 0 y se incrementa en 1 cada vez que el agente toma una acción.

- `self.k`: Es el atributo pivotal que re-define la función de distribución `X_s_a` que se utiliza para elegir una acción. `self.k` es la función creciente `f(x) = 1 + log10(delta_t)`, con `delta_t = 0` vale `1` y aumenta en orden logarítmico. Esto último es una decisión de diseño para beneficiar a la exploración en etapas iniciales. Queremos que el agente explore el entorno y no se quede estancado en un mínimo local, por lo que le damos más peso a la exploración en etapas iniciales, y a medida que el agente va aprendiendo, le damos más peso a la explotación.

- `self.Q`: Una tabla Q inicializada como una matriz de ceros. Las dimensiones de la matriz dependen del número de estados (`entorno.observation_space.n`) y el número de acciones (`entorno.action_space.n`) en el entorno en el que se encuentra el agente.

##### **Métodos**:

- `__init__(self, entorno) -> None`: El constructor de la clase que inicializa las variables de instancia.

- `elegir_accion(self, estado, max_accion) -> int`: Define la función de distribución `X_s_a` para el valor de `self.K` actual, tal cual presentada en el libro del curso. Luego, utilizando el metodo `np.random.choice` elige una acción de acuerdo a la función de distribución `X_s_a` y la devuelve.

- `aprender(self, estado_anterior, estado_siguiente, accion, recompensa)`: Este método actualiza la tabla Q utilizando la ecuación de actualización (Bellman). Aquí `Q_max` es el valor máximo de Q para el `estado_siguiente`. La tabla Q para el `estado_anterior` y la `accion` tomada se actualiza considerando la `recompensa` recibida y el valor descontado de `Q_max`.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Mitchell 97. Chapter 13
class AgenteRL(Agente):
    def __init__(self, entorno) -> None:
        super().__init__()
        self.gamma = 0.9
        self.k = 1
        self.Q = np.zeros((entorno.observation_space.n, entorno.action_space.n))
    
    
    def elegir_accion(self, estado, max_accion) -> int:
        # Exploracion vs Explotacion, Mitchell 97. p.379
        aux = np.power(np.ones(max_accion) * self.k , self.Q[estado])
        X_s_a = aux / np.sum(aux)
        
        # Elegimos una accion con distribucion X_s_a
        return np.random.choice(max_accion, 1, p= X_s_a)[0]
    
    def aprender(self, estado_anterior, estado_siguiente, accion, recompensa):
        # Aumento k (factor de "confianza")
        self.k += 1
        # Actualizamos la tabla Q con la ecuacion de Bellman 
        Q_max_estado_siguiente = np.max(self.Q[estado_siguiente])
        self.Q[estado_anterior, accion] = recompensa + self.gamma * Q_max_estado_siguiente

In [ ]:
# Funciones auxiliares

def metricas(iteraciones, penalizaciones):
  print(f"[ITERACIONES]       mean: {np.mean(iteraciones)}, std: {np.std(iteraciones)}")
  print(f"[PENALIZACIONES]    mean: {np.mean(penalizaciones)}, std: {np.std(penalizaciones)}")
  
def dibujar_subgrafico(episodios, datos, subindice, etiqueta_y, titulo, color='blue', escala_log_x=False, escala_log_y=False):
    plt.subplot(3, 1, subindice)
    plt.plot(episodios, datos, color=color)
    
    plt.xlabel('Episodio')
    plt.ylabel(etiqueta_y)
    plt.title(titulo)
    
    if escala_log_x: plt.xscale('log')
    if escala_log_y: plt.yscale('log')

    plt.grid(True)

## Parte 2

Analizar los resultados de una ejecución de mil episodios con el agente programado. Agregar un nuevo bloque de de texto discutiendo los resultados obtenidos

In [ ]:
agente = AgenteRL(entorno)
semilla = 1
num_iteraciones_episodios = []
num_penalizaciones_episodios = []
marcos_episodios = []

for i in range(1000):
    num_iteraciones, penalizaciones, marcos = ejecutar_episodio(agente, semilla)
    num_iteraciones_episodios += [num_iteraciones]
    num_penalizaciones_episodios += [penalizaciones]
    marcos_episodios += [marcos]

### Analisis de resultados

En nuestro algoritmo, hemos implementado una **estrategia de exploración que evoluciona a lo largo del tiempo**, lo que ha demostrado ser efectivo en el proceso de aprendizaje por refuerzo. Inicialmente, favorecemos la exploración, al ponderar la selección de acciones en función del conocimiento actual del agente y variando el parámetro `k` con el número de iteraciones. Esto permite al agente realizar exploración intensiva al principio y luego cambiar gradualmente hacia la explotación de conocimientos adquiridos. 

Los vestigios de exploración se pueden ver en los "picos" iniciales en la cantidad de iteraciones por episodio, esto da cuenta de la exploración intensiva que realiza el agente al principio. A medida que el agente acumula conocimiento sobre el entorno, la cantidad de iteraciones por episodio disminuye, llegado a un punto en el tiempo donde el agente se vuelve más eficiente en su toma de decisiones y la cantidad de iteraciones por episodio se estabiliza, que además coincide con la convergencia de la política óptima.

En nuestros resultados se puede observar que la **cantidad de penalizaciones disminuye** a medida que aumenta el número de episodios. Este comportamiento es esperado, ya que el agente va acumulando conocimiento sobre el entorno y aprende a evitar las acciones que conllevan penalizaciones. Además, el agente se vuelve más eficiente en su toma de decisiones, lo que se refleja en una disminución en la cantidad de iteraciones necesarias para alcanzar su objetivo.

En **comparación con el agente aleatorio**, el agente de aprendizaje por refuerzo logra reducir considerablemente la cantidad de iteraciones requeridas para alcanzar sus objetivos. Esto subraya la eficacia del aprendizaje por refuerzo en la mejora del desempeño del agente y su capacidad para tomar decisiones más informadas a lo largo del tiempo.

Para poder apreciar mejor la evolución del agente, se puede ejecutar el siguiente código que muestra una serie de episodios de interés.

*Nota*:

Dependiendo de la semilla utilizada en la clase, se pueden apreciar pequeñas variaciones post convergencia. Esto se debe a que a cada accion se le asigna una probabilidad no nula, y aun siendo muy pequeña, existe la posibilidad de que el agente tome una accion aleatoria en lugar de la accion optima. Esto se corrige cambiando la funcion que define `self.k` a `f(x) = 1 + sqrt(delta_t)`, es decir aumentando el orden de crecimiento de la funcion.

In [ ]:
metricas(num_iteraciones_episodios, num_penalizaciones_episodios)

plt.figure(figsize=(8, 12))

dibujar_subgrafico(
  [i for i in range(0, len(num_iteraciones_episodios))], 
  num_iteraciones_episodios, 
  1, 
  'Iteraciones', 
  'Iteraciones por episodio (escala logarítmica x,y)', 
  escala_log_x=True, 
  escala_log_y=True
)

dibujar_subgrafico(
  [i for i in range(0, len(num_penalizaciones_episodios))],
  num_penalizaciones_episodios,
  2,
  'Penalizaciones',
  'Penalizaciones por episodio (escala logarítmica x)',
  escala_log_x=True,
  color='red',
)

plt.subplots_adjust(hspace=0.5)
plt.show()

## Parte 3

Ejecutar 1000 episodios con una semilla diferente y analizar los resultados. Agregar un nuevo bloque de texto discutiendo los resultados obtenidos.

### Metodologia

En el código anterior, el ambiente en el cual nuestro agente realiza sus tareas, formalmente, es un **DMDP, Deterministic Markov Decision Process**. Esto significa que dado un estado y una acción, el ambiente siempre responde de la misma manera. Esto se modela fácilmente en el código de la sección anterior al mantener invariante la semilla.

Ahora, si cambiamos la semilla, el ambiente puede responder de diferentes maneras, dado que la composición del entorno cambia.

En particular, **la posición del taxi y las paradas de origen/destino son aleatorias** (aunque no su posición en la grilla); todo lo demás se mantiene constante.

Proponemos entonces, para analizar el comportamiento del agente variando la semilla, tomando la semilla inicial definir el par (semilla inicial (SI), semilla alternativa (SA)) de forma tal que:

- (SI, SA) difieren solo en la posición del taxi.
- (SI, SA) difieren en la posición del taxi y en la posición del origen.
- (SI, SA) difieren en la posición del taxi y en la posición del destino.

Tomando la siguiente proporción de ejecuciones:

- 60% con la misma semilla (SI). Para cada una de las variantes
  - 40% con una semilla alternativa (SA) que difiera solo en la posición del taxi.
  - 40% con una semilla alternativa (SA) que difiera en la posición del taxi y en la posición del destino.
  - 40% con una semilla alternativa (SA) que difiera en la posición del taxi, en la posición del origen y en la posición del destino. Es decir, en todo.

De esta forma podemos llevar el agente de un entorno conocido a uno ligéramente diferente, y luego a uno completamente diferente.

In [ ]:
metodologia = {
  'taxi': (1, 9),
  'taxi_destino': (1, 18),
  'taxi_origen_destino': (1, 7),
}

entorno = gym.make("Taxi-v3", render_mode='ansi').env

for metodo in metodologia.items():
  print(f"Metodologia: {metodo[0]}")
  for seed in metodo[1]:
    entorno.reset(seed = seed)
    print(f'SEED {seed}')
    print_env(entorno)
  print('~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')

In [ ]:
for metodo in metodologia.items():
  agente = AgenteRL(entorno)
  num_iteraciones_episodios = []
  num_penalizaciones_episodios = []
  
  for i in range(600):
    entorno.reset(seed = metodo[1][0])
    num_iteraciones, penalizaciones, _ = ejecutar_episodio(agente, metodo[1][0])
    num_iteraciones_episodios += [num_iteraciones]
    num_penalizaciones_episodios += [penalizaciones]
  
  for i in range(400):
    entorno.reset(seed = metodo[1][1])
    num_iteraciones, penalizaciones, _ = ejecutar_episodio(agente, metodo[1][1])
    num_iteraciones_episodios += [num_iteraciones]
    num_penalizaciones_episodios += [penalizaciones]
  
  print(f"Metodologia: {metodo[0]}")
  metricas(num_iteraciones_episodios, num_penalizaciones_episodios)
  plt.figure(figsize=(15,10))

  dibujar_subgrafico(
    [i for i in range(0, len(num_iteraciones_episodios))], 
    num_iteraciones_episodios, 
    1, 
    'Iteraciones', 
    'Iteraciones por episodio (escala logarítmica y)',
    escala_log_y=True
  )

  dibujar_subgrafico(
    [i for i in range(0, len(num_penalizaciones_episodios))],
    num_penalizaciones_episodios,
    2,
    'Penalizaciones',
    'Penalizaciones por episodio',
    color='red',
  )

  plt.subplots_adjust(hspace=0.5)
  plt.show()

### Analisis de resultados

Hay una correlación directa entre la cantidad de iteraciones por episodio y la cantidad de penalizaciones. Como sospechabamos, el agente se comporta de manera diferente en cada entorno, y esto se ve reflejado en la cantidad de iteraciones por episodio y la cantidad de penalizaciones.

Mientras mas "familiar" sea el entorno, menos iteraciones por episodio y menos penalizaciones se observan (de hecho, no se registran), converge mas rapido a la nueva politica optima.

De todas formas, el agente es capaz de aprender en entornos completamente desconocidos. Esto se ve reflejado en la disminución de la cantidad de iteraciones por episodio y la cantidad de penalizaciones a medida que el agente aprende. Hay unas iteraciones iniciales (alrededor de 600) donde el agente se comporta de manera errática, pero luego converge a la politica optima.

Lo que si no parece ser una diferencia significativa es el hecho de conservar una de las dos paradas de la semilla original, se desempeñan de forma similar en ambos casos.

## Parte 4

Realizar los cambios necesarios para que el agente sea capaz de tener un buen desempeño utilizando una semilla arbitraria, ejecutar iteraciones con semillas arbitrarias y analizar los resultados. Agregar un nuevo bloque de texto discutiendo los resultados obtenidos.

In [ ]:
import numpy as np
import pdb

# Mitchell 97. Chapter 13
class AgenteRL2(AgenteRL):
    def __init__(self, entorno) -> None:
        super().__init__(entorno)
        self.visits = {}
    
    def elegir_accion(self, estado, max_accion) -> int:
        return super().elegir_accion(estado, max_accion)
        
    def aprender(self, estado_anterior, estado_siguiente, accion, recompensa):
        # Actualizamos la tabla Q con la ecuacion de Bellman 
        Q_max_estado_siguiente = np.max(self.Q[estado_siguiente])
        classic_update = recompensa + self.gamma * Q_max_estado_siguiente
        tupla = (estado_anterior, accion)
        
        self.visits.setdefault(tupla, 0)
        self.visits[tupla] += 1
        self.k = self.visits[tupla]
        
        alpha_n =  1 / (1 + self.visits[tupla])
        
        self.Q[estado_anterior, accion] = (1 - alpha_n) * self.Q[estado_anterior, accion] + \
                                          (alpha_n) * classic_update

In [ ]:
agente_1 = AgenteRL(entorno)
agente_2 = AgenteRL2(entorno)
semilla = 50826476
num_iteraciones_episodios = [[], []]
num_penalizaciones_episodios = [[], []]

j = 0

for i in range(1000):
  if i < 600:
    j = i
  else:
    j = semilla + (i % 20)
    
  num_iteraciones, penalizaciones, _ = ejecutar_episodio(agente_2, j)
  num_iteraciones_episodios[0] += [num_iteraciones]
  num_penalizaciones_episodios[0] += [penalizaciones]
  
  num_iteraciones, penalizaciones, _ = ejecutar_episodio(agente_1, j)
  num_iteraciones_episodios[1] += [num_iteraciones]
  num_penalizaciones_episodios[1] += [penalizaciones]

metricas(num_iteraciones_episodios[0], num_penalizaciones_episodios[0])
metricas(num_iteraciones_episodios[1], num_penalizaciones_episodios[1])

plt.figure(figsize=(15,10))

dibujar_subgrafico(
  [i for i in range(0, len(num_iteraciones_episodios[0]))], 
  num_iteraciones_episodios[0], 
  1, 
  'Iteraciones', 
  'Iteraciones por episodio (escala logarítmica y)',
  escala_log_y=True
)

dibujar_subgrafico(
  [i for i in range(0, len(num_penalizaciones_episodios[0]))],
  num_penalizaciones_episodios[0],
  2,
  'Penalizaciones',
  'Penalizaciones por episodio',
  color='red',
)

plt.subplots_adjust(hspace=0.5)
plt.show()

plt.figure(figsize=(15,10))

dibujar_subgrafico(
  [i for i in range(0, len(num_iteraciones_episodios[1]))], 
  num_iteraciones_episodios[1], 
  1, 
  'Iteraciones', 
  'Iteraciones por episodio (escala logarítmica y)',
  escala_log_y=True
)

dibujar_subgrafico(
  [i for i in range(0, len(num_penalizaciones_episodios[1]))],
  num_penalizaciones_episodios[1],
  2,
  'Penalizaciones',
  'Penalizaciones por episodio',
  color='red',
)

plt.subplots_adjust(hspace=0.5)
plt.show()